Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 967.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 66.9 MB/s eta 0:00:00


Örnek: Bell Durumu $\ket{B_{00}}$’in Pauli Gösterimi ile Dolaşıklık Kontrolü


Cirq

In [2]:
import cirq
import numpy as np

# Pauli matrisleri
I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])

paulis = [I, X, Y, Z]

# Bell durumu |B00>
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0,q1)
)
simulator = cirq.Simulator()
result = simulator.simulate(circuit)
psi = result.final_state_vector

# Yoğunluk matrisi
rho = np.outer(psi, np.conj(psi))

# Pauli katsayılarını hesapla
c = np.zeros((4,4), dtype=complex)
for i in range(4):
    for j in range(4):
        c[i,j] = np.trace(rho @ np.kron(paulis[i], paulis[j]))

print("Pauli katsayıları c_ij:")
print(np.round(c.real, 3))

# Dolaşıklık kriteri
criterion = abs(c[0,0]) + abs(c[1,1]) + abs(c[2,2]) + abs(c[3,3])
if criterion <= 1:
    print("Cirq: Sistem ayrılabilir (dolaşık değil).")
else:
    print("Cirq: Sistem dolaşık.")


Pauli katsayıları c_ij:
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  1.]]
Cirq: Sistem dolaşık.


Qiskit

In [3]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
import numpy as np

# Pauli matrisleri
I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
paulis = [I, X, Y, Z]

# Devre ve durum vektörü
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0,1)
psi = Statevector.from_instruction(qc).data

# Yoğunluk matrisi
rho = np.outer(psi, np.conj(psi))

# Pauli katsayıları
c = np.zeros((4,4), dtype=complex)
for i in range(4):
    for j in range(4):
        c[i,j] = np.trace(rho @ np.kron(paulis[i], paulis[j]))

print("Pauli katsayıları c_ij:")
print(np.round(c.real,3))

# Dolaşıklık kriteri
criterion = abs(c[0,0]) + abs(c[1,1]) + abs(c[2,2]) + abs(c[3,3])
if criterion <= 1:
    print("Qiskit: Sistem ayrılabilir (dolaşık değil).")
else:
    print("Qiskit: Sistem dolaşık.")


Pauli katsayıları c_ij:
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  1.]]
Qiskit: Sistem dolaşık.


QuTiP

In [4]:
from qutip import tensor, basis, cnot, hadamard_transform
import numpy as np

# Pauli matrisleri
I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
paulis = [I,X,Y,Z]

# Bell durumu |B00>
psi0 = tensor(basis(2,0), basis(2,0))
H = hadamard_transform()
psi1 = tensor(H*psi0.ptrace(0), psi0.ptrace(1))
psi_final = cnot() * psi1

# Yoğunluk matrisi
rho = psi_final * psi_final.dag()
rho_mat = rho.full()

# Pauli katsayıları
c = np.zeros((4,4), dtype=complex)
for i in range(4):
    for j in range(4):
        c[i,j] = np.trace(rho_mat @ np.kron(paulis[i], paulis[j]))

print("Pauli katsayıları c_ij:")
print(np.round(c.real,3))

# Dolaşıklık kriteri
criterion = abs(c[0,0]) + abs(c[1,1]) + abs(c[2,2]) + abs(c[3,3])
if criterion <= 1:
    print("QuTiP: Sistem ayrılabilir (dolaşık değil).")
else:
    print("QuTiP: Sistem dolaşıktır.")


ImportError: cannot import name 'cnot' from 'qutip' (/usr/local/lib/python3.12/dist-packages/qutip/__init__.py)